In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import itertools
import tensorflow as tf
import pandas as pd

import tensorflow
from tensorflow.keras import backend as K
from utility import LoadDataIot
from Model_cnn import DCNet_Dropouts   #DCNet_Dropouts
import random
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
import numpy as np
#import tensorflow_addons as tf
import tensorflow
from sklearn.utils import shuffle
#from tensorflow_addons.optimizers import extend_with_decoupled_weight_decay
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
np_config.enable_numpy_behavior()


In [2]:
!python dataset/Class_filter.py

2023-09-17 02:56:58.331826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
256 249 260 253 249 236 251 235 261 259
(3180, 500)
(3180,)
[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6
 7 8 9 0 1 2 3 4 5 6 7 8 9]
(1272, 500)
Data dimensions:
X: Training data's shape :  (890, 500)
X: Validating data's shape :  (229, 500)
X: Testing data's shape :  (153, 500)
X: open data's shape :  (1908, 500)


In [3]:
# Training the DF model
NB_EPOCH = 500   # Number of training epoch
print("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 64 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 500 # Packet sequence length
NB_CLASSES = 4 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)
Lambda=1
alpha=1

random.seed(0)

def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]


def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / np.sum(Precision_Differences)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / np.sum(Recall_Differences)

    # print('Precision_Differences_Per',Precision_Differences_Per)
    # print('Recall_Differences_Per',Recall_Differences_Per)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = (Matrix[k][k] / np.sum(Matrix, axis=1)[k])  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / np.sum(Matrix, axis=0)[k]
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / np.sum(Matrix, axis=1)[k]

    Recall = np.average(Recalls)
    print("Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

OPTIMIZER = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.0001) # Optimizer


# Data: shuffled and split between train and test sets
print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_test, y_test, X_valid, y_valid  = LoadDataIot()

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')
y_test = y_test.astype('float32')
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]


y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

# Building and training model
print("Building and training DC model")

model = DCNet_Dropouts.build(input_shape=INPUT_SHAPE, nb_classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

filepath = 'DC_without_norm.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# callbacks_list.append(EarlyStopping(monitor='val_acc', mode='max', patience=6))

model.summary()

print(X_train.shape)
print(y_train.shape)

history = model.fit(X_train, y_train,batch_size=BATCH_SIZE, epochs=NB_EPOCH,verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)


with open('/content/training_history/His_Dict_without_Nom', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)


print("############## Training is Done Successfully ###################")
#model.save('DC_normal_model.hdf5')
#model=load_model('AWF.hdf5')

# Start evaluating model with testing data
score_test = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/DC_without_softmax.hdf5')

del model


Number of Epoch:  500
Loading and preparing data for training, and evaluating the model
Loading non-defended dataset for closed-world scenario
Data dimensions:
X: Training data's shape :  (890, 500)
y: Training data's shape :  (890,)
X: Validation data's shape :  (229, 500)
y: Validation data's shape :  (229,)
X: Testing data's shape :  (153, 500)
y: Testing data's shape :  (153,)
Building and training DC model
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv1D)              (None, 485, 32)           544       
                                                                 
 conv2 (Conv1D)              (None, 470, 32)           16416     
                                                                 
 conv3 (Conv1D)              (None, 455, 32)           16416     
                                                                 
 cov_dropout (Dropout)       (None, 4

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 - 0s - loss: 30.7108 - accuracy: 0.4438 - val_loss: 0.9706 - val_accuracy: 0.7991 - 177ms/epoch - 13ms/step
Epoch 3/500

Epoch 3: val_accuracy improved from 0.79913 to 0.89956, saving model to DC_without_norm.hdf5
14/14 - 0s - loss: 15.4803 - accuracy: 0.5270 - val_loss: 0.2641 - val_accuracy: 0.8996 - 175ms/epoch - 12ms/step
Epoch 4/500

Epoch 4: val_accuracy improved from 0.89956 to 0.90393, saving model to DC_without_norm.hdf5
14/14 - 0s - loss: 10.0974 - accuracy: 0.5854 - val_loss: 0.2317 - val_accuracy: 0.9039 - 181ms/epoch - 13ms/step
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.90393
14/14 - 0s - loss: 8.0463 - accuracy: 0.6112 - val_loss: 0.2728 - val_accuracy: 0.9039 - 137ms/epoch - 10ms/step
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.90393
14/14 - 0s - loss: 5.2902 - accuracy: 0.6180 - val_loss: 0.2699 - val_accuracy: 0.8908 - 155ms/epoch - 11ms/step
Epoch 7/500

Epoch 7: val_accuracy did not improve from 0.90393
14/14 - 0s - loss: 4.1095 - 

Testing closed accuracy_without_norm: 0.9934640526771545


In [5]:
################## Calculating mean anchor vectors for each class ######################

tflite_model_predictions = model_penultimate.predict(X_train)

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name]=np.array([0] * NB_CLASSES)

y_train=np.argmax(y_train,axis=1)
y_valid=np.argmax(y_valid,axis=1)
y_test=np.argmax(y_test,axis=1)

for i in range(len(tflite_model_predictions)):
    if (y_train[i] == 0 and np.argmax(tflite_model_predictions[i]) == y_train[i]):
        Mean_0 = np.sum([Mean_0, tflite_model_predictions[i]], axis=0)

    elif (y_train[i] == 1 and np.argmax(tflite_model_predictions[i]) == y_train[i]):
        Mean_1 = np.sum([Mean_1, tflite_model_predictions[i]], axis=0)

    elif (y_train[i] == 2 and np.argmax(tflite_model_predictions[i]) == y_train[i]):
        Mean_2 = np.sum([Mean_2, tflite_model_predictions[i]], axis=0)

    elif (y_train[i] == 3 and np.argmax(tflite_model_predictions[i]) == y_train[i]):
        Mean_3 = np.sum([Mean_3, tflite_model_predictions[i]], axis=0)

    elif (y_train[i] == 4 and np.argmax(tflite_model_predictions[i]) == y_train[i]):
        Mean_4 = np.sum([Mean_4, tflite_model_predictions[i]], axis=0)

    elif (y_train[i] == 5 and np.argmax(tflite_model_predictions[i]) == y_train[i]):
        Mean_5 = np.sum([Mean_5, tflite_model_predictions[i]], axis=0)

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name] = locals()[variable_name]/list(y_train).count(i)


Mean_vectors = np.array([Mean_0, Mean_1, Mean_2, Mean_3])
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)


############## Calculating the 90% Thresholds for each method ##################


tflite_model_predictions = model_penultimate.predict(X_valid)

Dis_Values = {}

for i in range(NB_CLASSES):
    Dis_Values[i] = []

for i in range(len(tflite_model_predictions)):
    if (y_valid[i] == 0 and np.argmax(tflite_model_predictions[i]) == y_valid[i]):
        dist = np.linalg.norm(Mean_0 - tflite_model_predictions[i])
        Dis_Values[0].append(dist)

    elif (y_valid[i] == 1 and np.argmax(tflite_model_predictions[i]) == y_valid[i]):
        dist = np.linalg.norm(Mean_1 - tflite_model_predictions[i])
        Dis_Values[1].append(dist)

    elif (y_valid[i] == 2 and np.argmax(tflite_model_predictions[i]) == y_valid[i]):
        dist = np.linalg.norm(Mean_2 - tflite_model_predictions[i])
        Dis_Values[2].append(dist)

    elif (y_valid[i] == 3 and np.argmax(tflite_model_predictions[i]) == y_valid[i]):
        dist = np.linalg.norm(Mean_3 - tflite_model_predictions[i])
        Dis_Values[3].append(dist)

    elif (y_valid[i] == 4 and np.argmax(tflite_model_predictions[i]) == y_valid[i]):
        dist = np.linalg.norm(Mean_4 - tflite_model_predictions[i])
        Dis_Values[4].append(dist)

    elif (y_valid[i] == 5 and np.argmax(tflite_model_predictions[i]) == y_valid[i]):
        dist = np.linalg.norm(Mean_5 - tflite_model_predictions[i])
        Dis_Values[5].append(dist)

percentile = 0.9
for i in range(NB_CLASSES):
    Dis_Values[i].sort()
    variable_name = f"Th_{i}"
    locals() [variable_name] = Dis_Values[i][int(len(Dis_Values[i]) * percentile)]

Threasholds_1 = np.array([Th_0, Th_1, Th_2, Th_3])
np.save('/content/temp_variables/Threasholds_1.npy', Threasholds_1)

print("Thresholds are calulated for method 1")

############################################################################################
Dis_Values = {}

for i in range(NB_CLASSES):
    Dis_Values[i] = []

for i in range(len(tflite_model_predictions)):
    if (y_valid[i] == 0):
        dist = np.linalg.norm(Mean_vectors[0] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 0:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i]) - dist
        Dis_Values[0].append(Tot)


    elif (y_valid[i] == 1):
        dist = np.linalg.norm(Mean_vectors[1] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 1:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i]) - dist
        Dis_Values[1].append(Tot)

    elif (y_valid[i] == 2):
        dist = np.linalg.norm(Mean_vectors[2] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 2:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i]) - dist
        Dis_Values[2].append(Tot)

    elif (y_valid[i] == 3):
        dist = np.linalg.norm(Mean_vectors[3] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 3:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i]) - dist
        Dis_Values[3].append(Tot)

percentile = 0.9
for i in range(NB_CLASSES):
    Dis_Values[i].sort()
    variable_name = f"Th_{i}"
    locals() [variable_name] = Dis_Values[i][int(len(Dis_Values[i]) * percentile)]

Threasholds_2 = np.array([Th_0, Th_1, Th_2, Th_3])
np.save('/content/temp_variables/Threasholds_2.npy', Threasholds_2)

print("Thresholds are calulated for method 2")

############################################################################################
Dis_Values = {}

for i in range(NB_CLASSES):
    Dis_Values[i] = []

for i in range(len(tflite_model_predictions)):
    if (y_valid[i] == 0):
        dist = np.linalg.norm(Mean_vectors[0] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 0:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i])
        Dis_Values[0].append(dist / Tot)


    elif (y_valid[i] == 1):
        dist = np.linalg.norm(Mean_vectors[1] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 1:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i])
        Dis_Values[1].append(dist / Tot)

    elif (y_valid[i] == 2):
        dist = np.linalg.norm(Mean_vectors[2] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 2:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i])
        Dis_Values[2].append(dist / Tot)

    elif (y_valid[i] == 3):
        dist = np.linalg.norm(Mean_vectors[3] - tflite_model_predictions[i])
        Tot = 0
        for k in range(NB_CLASSES):
            if k != 3:
                Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i])
        Dis_Values[3].append(dist / Tot)

percentile = 0.9
for i in range(NB_CLASSES):
    Dis_Values[i].sort()
    variable_name = f"Th_{i}"
    locals() [variable_name] = Dis_Values[i][int(len(Dis_Values[i]) * percentile)]

Threasholds_3 = np.array([Th_0, Th_1, Th_2, Th_3])
np.save('/content/temp_variables/Threasholds_3.npy', Threasholds_3)

print("Thresholds are calulated for method 3")

8/8 [==============================] - 0s 31ms/step
Thresholds are calulated for method 1
Thresholds are calulated for method 2
Thresholds are calulated for method 3


In [6]:
############################################################################################

######################## Evaluating results ##########################
dataset_dir="/content/dataset/"
X_open = np.load(dataset_dir+'X_open.npy')
y__open = np.load(dataset_dir+'y_open.npy')
y_open=np.array([NB_CLASSES]*len(y__open))

X_open = X_open.astype('float32')
y_open = y_open.astype('int8')

X_open = X_open[:, :,np.newaxis]

tflite_model_predictions = model_penultimate.predict(X_test)
tflite_model_predictions_open = model_penultimate.predict(X_open)

#############################################################################################################
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):

    d = np.argmax(tflite_model_predictions[i], axis=0)
    if np.linalg.norm(tflite_model_predictions[i] - Mean_vectors[d]) > Threasholds_1[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)


prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):

    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i] - Mean_vectors[d]) > Threasholds_1[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):
    d = np.argmax(tflite_model_predictions[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i]) - dist

    if Tot < Threasholds_2[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):
    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions_open[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions_open[i]) - dist

    if Tot < Threasholds_2[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):
    d = np.argmax(tflite_model_predictions[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i])

    Tot = dist / Tot
    if Tot > Threasholds_3[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):
    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions_open[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions_open[i])

    Tot = dist / Tot
    if Tot > Threasholds_3[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

60/60 [==============================] - 0s 4ms/step

############## Distance Method 1 #################################

Test accuracy Normal model_Closed_set : 0.9084967320261438
Test accuracy Normal model_Open_set : 0.9318658280922432

Precision:  0.6189065852700293
Recall:  0.9120323886639676
Average novel F1_Score:  0.7374073784093017

Micro Precision:  0.516728624535316
Micro Recall:  0.9084967320261438
Average Micro F1_Score:  0.6587677678896477

Precision: 0.8006134969325154
Recall: 0.9089068825910931
Average Macro F1_Score:  0.8513301416195094

############## Distance Method 2 #################################

Test accuracy Normal model_Closed_set : 0.10457516339869281
Test accuracy Normal model_Open_set : 1.0

Precision:  1.0
Recall:  0.09834822275013663
Average novel F1_Score:  0.1790838639652273

Micro Precision:  1.0
Micro Recall:  0.10457516339869281
Average Micro F1_Score:  0.18934911071180982

Precision: 1.0
Recall: 0.10492577597840755
Average Macro F1_Score:  0.189923